# Feature Engineering Homework 
***
**Name**: Poorwa Hirve 

**Kaggle Username**: phirwe
***

This assignment is due on Moodle by **5pm on Friday February 23rd**. Additionally, you must make at least one submission to the **Kaggle** competition before it closes at **4:59pm on Friday February 23rd**. Submit only this Jupyter notebook to Moodle. Do not compress it using tar, rar, zip, etc. Your solutions to analysis questions should be done in Markdown directly below the associated question.  Remember that you are encouraged to discuss the problems with your instructors and classmates, but **you must write all code and solutions on your own**.  For a refresher on the course **Collaboration Policy** click [here](https://github.com/chrisketelsen/CSCI5622-Machine-Learning/blob/master/resources/syllabus.md#collaboration-policy)



## Overview 
***

When people are discussing popular media, there’s a concept of spoilers. That is, critical information about the plot of a TV show, book, or movie that “ruins” the experience for people who haven’t read / seen it yet.

The goal of this assignment is to do text classification on forum posts from the website [tvtropes.org](http://tvtropes.org/), to predict whether a post is a spoiler or not. We'll be using the logistic regression classifier provided by sklearn.

Unlike previous assignments, the code provided with this assignment has all of the functionality required. Your job is to make the functionality better by improving the features the code uses for text classification.

**NOTE**: Because the goal of this assignment is feature engineering, not classification algorithms, you may not change the underlying algorithm or it's parameters

This assignment is structured in a way that approximates how classification works in the real world: Features are typically underspecified (or not specified at all). You, the data digger, have to articulate the features you need. You then compete against others to provide useful predictions.

It may seem straightforward, but do not start this at the last minute. There are often many things that go wrong in testing out features, and you'll want to make sure your features work well once you've found them.


## Kaggle In-Class Competition 
***

In addition to turning in this notebook on Moodle, you'll also need to submit your predictions on Kaggle, an online tournament site for machine learning competitions. The competition page can be found here:  

[https://www.kaggle.com/c/feature-engineering-csci-5622-spring-2018](https://www.kaggle.com/c/feature-engineering-csci-5622-spring-2018)

Additionally, a private invite link for the competition has been posted to Piazza. 

The starter code below has a `model_predict` method which produces a two column CSV file that is correctly formatted for Kaggle (predictions.csv). It should have the example Id as the first column and the prediction (`True` or `False`) as the second column. If you change this format your submissions will be scored as zero accuracy on Kaggle. 

**Note**: You may only submit **THREE** predictions to Kaggle per day.  Instead of using the public leaderboard as your sole evaluation processes, it is highly recommended that you perform local evaluation using a validation set or cross-validation. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline 

### [25 points] Problem 1: Feature Engineering 
***

The `FeatEngr` class is where the magic happens.  In it's current form it will read in the training data and vectorize it using simple Bag-of-Words.  It then trains a model and makes predictions.  

25 points of your grade will be generated from your performance on the the classification competition on Kaggle. The performance will be evaluated on accuracy on the held-out test set. Half of the test set is used to evaluate accuracy on the public leaderboard.  The other half of the test set is used to evaluate accuracy on the private leaderboard (which you will not be able to see until the close of the competition). 

You should be able to significantly improve on the baseline system (i.e. the predictions made by the starter code we've provided) as reported by the Kaggle system.  Additionally, the top **THREE** students from the **PRIVATE** leaderboard at the end of the contest will receive 5 extra credit points towards their Problem 1 score.


In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from nltk.stem.porter import *
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from scipy.sparse import csr_matrix
import numpy as np
import re
import pandas as pd
import scipy as sp
import nltk
import string
import math

title = []
year = []
runtime = []
season = []
episodes = []
genre = []

title_year = dict()
for line in open('../data/spoilers/title_year.csv'):
    input_line = line.strip().split(',')
    if input_line[0] not in title_year.keys():
        title_year[input_line[0]] = round(float(input_line[1]), 3)
    
for line in open('../data/spoilers/tv_info.csv'):
    input_line = line.strip().split('***')
    if input_line[0] not in title:
        title.append(input_line[0])
        year.append(float(input_line[1]))
        runtime.append(float(input_line[2]))
        season.append(float(input_line[3]))
        episodes.append(float(input_line[4]))

# function most_common referred from:
# https://stackoverflow.com/questions/1518522/python-most-common-element-in-a-list
        
import itertools
import operator

def most_common(L):
    SL = sorted((x, i) for i, x in enumerate(L))
    groups = itertools.groupby(SL, key=operator.itemgetter(0))
    def _auxfun(g):
        item, iterable = g
        count = 0
        min_index = len(L)
        for _, where in iterable:
            count += 1
            min_index = min(min_index, where)
        return count, -min_index
    return max(groups, key=_auxfun)[0]

class num_of_words(BaseEstimator, TransformerMixin):
    def _init_(self):
        pass
    def fit(self, examples):
        return self
    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i=0
        for ex in examples:
            count = len(ex.split())
            features[i,0] = count
            i += 1
        return features

class adj_count(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        for ex in examples:
            text = word_tokenize(str(ex).translate(string.punctuation))
            mylist = nltk.pos_tag(text)
            count = 0
            for item in mylist:
                if item[1].lower() == 'jjs' or item[1].lower() == 'jjr':
                    count += 1
            features[i,0] = count
            i += 1
        return features
    
class adv_count(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        for ex in examples:
            text = word_tokenize(str(ex).translate(string.punctuation))
            mylist = nltk.pos_tag(text)
            count = 0
            for item in mylist:
                if item[1].lower() == 'rb':
                    count += 1
            features[i,0] = count
            i += 1
        return features
    
class capital_counts(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, examples):
        return self
    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        for ex in examples:
            features[i,0] = sum(1 for letter in str(ex) if letter.isupper())
            i += 1
        
        return features

class Year(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, examples):
        return self
    def transform(self, examples):

        features = np.zeros((len(examples),1))
        i = 0
        for ex in examples:
            try:
#                 features[i,0] = year[title.index(str(ex))]
                features[i,0] = title_year[str(ex)]
            except:
#                 features[i,0] = most_common(year)
                features[i,0] = 1.0
            i += 1
        
    
class Season(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, examples):
        return self
    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        for ex in examples:
            try:
                features[i, 0] = season[title.index(str(ex))]
            except:
                features[i, 0] = most_common(season)
            i += 1
        return features
    
class Runtime(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, examples):
        return self
    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        for ex in examples:
            try:
                features[i, 0] = runtime[title.index(str(ex))]
            except:
                features[i, 0] = most_common(runtime)
            i += 1
        return features
    
class Episodes(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, examples):
        return self
    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        for ex in examples:
            try:
                features[i, 0] = episodes[title.index(str(ex))]
            except:
                features[i, 0] = most_common(episodes)
            i += 1
        return features
    
def sentence_preprocess(examples):
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()
    new_examples = []
    for ex in examples:
        new_ex = [ps.stem(word) for word in word_tokenize(ex)]
        new_examples.append(' '.join([wnl.lemmatize(word) for word in new_ex]))
    return new_examples

def trope_preprocess(examples):
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()
    new_examples = []
    for ex in examples:
        # https://stackoverflow.com/questions/37505224/add-a-space-between-each-word-in-the-string
        ex = re.sub('([A-Z])', r' \1', ex).strip()
        new_ex = [wnl.lemmatize(word) for word in word_tokenize(ex)]
        new_examples.append(' '.join([ps.stem(word) for word in new_ex]))
    return new_examples

class FeatEngr:
    def __init__(self):

        from sklearn.feature_extraction.text import CountVectorizer
        from sklearn.feature_extraction.text import TfidfVectorizer
        self.sentence_vectorizer = FeatureUnion([
                                        ('num-of-words', num_of_words()),
                                        ('adj-count', adj_count()),
                                        ('adv-count', adv_count()),
                                        ('capital-count', capital_counts()),
                                        ('tfidf', TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2), lowercase=True,
                                          norm='l2'))])
        self.trope_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2), lowercase=True,
                                          norm='l2')
        self.year_vectorizer =  Year()
        self.season_vectorizer = Season()

    def build_train_features(self, examples, tropes, pages):
        """
        Method to take in training text features and do further feature engineering
        Most of the work in this homework will go here, or in similar functions
        :param examples: currently just a list of forum posts
        """
        prep_sentences = sentence_preprocess(examples)
        prep_tropes = trope_preprocess(tropes)

        sentence = self.sentence_vectorizer.fit_transform(prep_sentences)
        trope = self.trope_vectorizer.fit_transform(prep_tropes)
        year = self.year_vectorizer.fit_transform(pages)
        season = self.season_vectorizer.fit_transform(pages)

        vector = sp.sparse.hstack((sentence, trope, year, season))
        
        return csr_matrix(vector)

    def get_test_features(self, examples, tropes, pages):
        """
        Method to take in test text features and transform the same way as train features
        :param examples: currently just a list of forum posts
        """

        prep_sentences = sentence_preprocess(examples)
        prep_tropes = trope_preprocess(tropes)

        sentence = self.sentence_vectorizer.transform(prep_sentences)
        trope = self.trope_vectorizer.transform(prep_tropes)
        year = self.year_vectorizer.transform(pages)
        season = self.season_vectorizer.transform(pages)
        
        vector = sp.sparse.hstack((sentence, trope, year, season))
        return csr_matrix(vector)

    def show_top10(self):
        """
        prints the top 10 features for the positive class and the
        top 10 features for the negative class.
        """
        feature_names = np.asarray(self.vectorizer.get_feature_names())
        top10 = np.argsort(self.logreg.coef_[0])[-10:]
        bottom10 = np.argsort(self.logreg.coef_[0])[:10]
        print("Pos: %s" % " ".join(feature_names[top10]))
        print("Neg: %s" % " ".join(feature_names[bottom10]))


    def train_model(self, random_state=1234):
        """
        Method to read in training data from file, and
        train Logistic Regression classifier.

        :param random_state: seed for random number generator
        """

        from sklearn.linear_model import LogisticRegression
        import pandas as pd
        from sklearn.linear_model import LogisticRegression
        from sklearn.metrics import accuracy_score
        from sklearn.model_selection import KFold
        from scipy.sparse import csr_matrix

        # load data
        dfTrain = pd.read_csv("../data/spoilers/train.csv")

        # get training features and labels
        self.X_train = self.build_train_features(list(dfTrain["sentence"]), list(dfTrain["trope"]), list(dfTrain["page"]))
        self.y_train = np.array(dfTrain["spoiler"], dtype=int)
        
        self.logreg = LogisticRegression(random_state=random_state)
        self.logreg.fit(self.X_train, self.y_train)
        
        from sklearn.model_selection import KFold
        kf = KFold(n_splits=5, random_state=None, shuffle=True)
        avg_accuracy = []
        for train_index, test_index in kf.split(self.X_train):
            k_X, k_x = self.X_train[train_index], self.X_train[test_index]
            k_Y, k_y = self.y_train[train_index], self.y_train[test_index]
            sentences = [list(dfTrain["sentence"])[i] for i in test_index]
            tropes = [list(dfTrain["trope"])[i] for i in test_index]
            pages = [list(dfTrain["page"])[i] for i in test_index]

            self.logreg = LogisticRegression(random_state=random_state)
            self.logreg.fit(k_X, k_Y)
            k_y_pred = self.logreg.predict(k_x)
            print (accuracy_score(k_y, k_y_pred))
            
            self.error_analysis(k_x, k_y, k_y_pred, sentences, tropes, pages)

            # TODO: Error analysis
            avg_accuracy.append(accuracy_score(k_y, k_y_pred))
        print('Average Accuracy is: ', sum(avg_accuracy) / len(avg_accuracy))


    def model_predict(self):
        """
        Method to read in test data from file, make predictions
        using trained model, and dump results to file
        """

        # read in test data
        dfTest = pd.read_csv("../data/spoilers/test.csv")

        # featurize test data
        self.X_test = self.get_test_features(list(dfTest["sentence"]), list(dfTest['trope']), list(dfTest["page"]))

        # make predictions on test data
        pred = self.logreg.predict(self.X_test)

        # dump predictions to file for submission to Kaggle
        pd.DataFrame({"spoiler": np.array(pred, dtype=bool)}).to_csv("prediction.csv", index=True, index_label="Id")

        
    def error_analysis(self, train, test, pred, sentences, tropes, pages):
        pass
        from sklearn.metrics import confusion_matrix
        C = confusion_matrix(test, pred)
        print (C)
        top_s = []
        top_t = []
        top_p = []
        top_test = []
        top_pred = []
        for i in range(len(test)):
            if test[i] != pred[i] and i <= 5:
                top_s.append(sentences[i])
                top_t.append(tropes[i])
                top_p.append(pages[i])
                top_test.append(test[i])
                top_pred.append(pred[i])
                
        print (top_s)
        print (top_t)
        print (top_p)
        print (top_test)
        print (top_pred)

In [3]:
# Instantiate the FeatEngr clas 
feat = FeatEngr()

feat.train_model(random_state=1230)
feat.model_predict()

0.737677527151
[[816 299]
 [329 950]]
['The first episode deals with   one of the season 1 bad guys getting killed by an  Eldritch Abomination  with telekinesis!', "Sure it's off-screen and its costume looks fake and silly, but still!", 'Cheryl, Andy, and Dana make Jim think his dead friend came back to life, because Jim stole back his playing card that said friend stole the day before he died.']
['DarkerAndEdgier', 'DarkerAndEdgier', 'PracticalJoke']
['AaronStone', 'AaronStone', 'AccordingToJim']
[1, 1, 0]
[0, 0, 1]
0.747702589808
[[836 321]
 [283 954]]
["For example in one episode Jim explains that men and women's brains are wired so they won't remember certain things, Cheryl gets mad at him, only to later find herself trapped in the garage because none of the girls can remember the key code for the door, which her toddler son punches in effortlessly.", 'Andy has been a victim of this.']
['DoubleStandard', 'GroinAttack']
['AccordingToJim', 'AccordingToJim']
[0, 0]
[1, 1]
0.7222222222

### [25 points] Problem 2: Motivation and Analysis 
***

The job of the written portion of the homework is to convince the grader that:

- Your new features work
- You understand what the new features are doing
- You had a clear methodology for incorporating the new features

Make sure that you have examples and quantitative evidence that your features are working well. Be sure to explain how you used the data (e.g., did you have a validation set? did you do cross-validation?) and how you inspected the results. In addition, it is very important that you show some kind of an **error analysis** throughout your process.  That is, you should demonstrate that you've looked at misclassified examples and put thought into how you can craft new features to improve your model. 

A sure way of getting a low grade is simply listing what you tried and reporting the Kaggle score for each. You are expected to pay more attention to what is going on with the data and take a data-driven approach to feature engineering.

---
___

# Solution:
---


## Cross Validation:
- I first manually divided the training data into 5 folds, running the classifier 5 times.
- Next, I decided to use sklearn's KFold for cross validation

## Preprocessing:
- I stemmed and lemmatized sentences and tropes. Preprocessing brought the accuracy up.

## Feature Selection:

### N-grams
- At first, I selected a trigram model. On increasing the folds to 10, I noticed that the trigram model did not work that well.
- Next I decided to use bigrams. Its performance was better than trigrams.

### Length of the Sentence
- Upon looking into the 'train.csv' file, I discovered that the more the length of the sentences, the more likely it was to be a spoiler.

### Parts of Speech
- Upon error analysis of the mislabelled examples, that is printing the sentences that were mislabelled, I noticed that sentences containing spoilers had more adjectives and adverbs, that is with parts of speech 'JJR', 'JJS' and 'RB'.

### Capital Letters
- Most spoilers contain character names or episode titles, which have many capital letters that influence the decision.

### TfIdf
- This is a feature to decide how important a word is to a document in a collection. Bigrams were used along with the TfidfVectorizer for sentences as well as tropes.

### Years, Seasons
- The more recent TV shows would be more likely to contain spoilers, and shows that long running shows would have stories that develop and have spoilers.

## Results

I achieved an average accuracy of 73% on Cross Validation with 5 folds.
On Kaggle, I achieved top 2 accuracies of 72.72% and 72.31%.


## Features and their effect on the accuracy

| Feature                | Effect on Accuracy |
|------------------------|----------------------|
|Count of Quotation Marks| Decreased            |
|Count of Proper Nouns   | Decreased            |
|5-grams | Decreased |
|Trigrams                | Decreased            |
|Bigrams                 | Increased            |
|Year (difference from now)|Decreased           |
|Year (Normalizing) | Increased |
|Length of Sentence | Increased |
|Capital Letters | Increased |
|Tfidf | Increased |
|Seasons | Increased |
|Runtime | Decreased |


## Data From IMDb

I obtained the TV series information from IMDb using imdbpy and stored it all in a file called 'tv_info.csv'.

---
___

## Error Analysis

The error_analysis function shows some of the misclassified examples and what they were misclassified as. I made use of that function to find out which features had more impact. On observing the misclassified sentences and the training data corresponding to them, I was able to improve the model by trying out different features. I also used a confusion matrix for error analysis.






### Hints 
***

- Don't use all the data until you're ready. 

- Examine the features that are being used.

- Do error analyses.

- If you have questions that aren’t answered in this list, feel free to ask them on Piazza.

### FAQs 
***

> Can I heavily modify the FeatEngr class? 

Totally.  This was just a starting point.  The only thing you cannot modify is the LogisticRegression classifier.  

> Can I look at TV Tropes?

In order to gain insight about the data yes, however, your feature extraction cannot use any additional data (beyond what I've given you) from the TV Tropes webpage.

> Can I use IMDB, Wikipedia, or a dictionary?

Yes, but you are not required to. So long as your features are fully automated, they can use any dataset other than TV Tropes. Be careful, however, that your dataset does not somehow include TV Tropes (e.g. using all webpages indexed by Google will likely include TV Tropes).

> Can I combine features?

Yes, and you probably should. This will likely be quite effective.

> Can I use Mechanical Turk?

That is not fully automatic, so no. You should be able to run your feature extraction without any human intervention. If you want to collect data from Mechanical Turk to train a classifier that you can then use to generate your features, that is fine. (But that’s way too much work for this assignment.)

> Can I use a Neural Network to automatically generate derived features? 

No. This assignment is about your ability to extract meaningful features from the data using your own experimentation and experience.

> What sort of improvement is “good” or “enough”?

If you have 10-15% improvement over the baseline (on the Public Leaderboard) with your features, that’s more than sufficient. If you fail to get that improvement but have tried reasonable features, that satisfies the requirements of assignment. However, the extra credit for “winning” the class competition depends on the performance of other students.

> Where do I start?  

It might be a good idea to look at the in-class notebook associated with the Feature Engineering lecture where we did similar experiments. 


> Can I use late days on this assignment? 

You can use late days for the write-up submission, but the Kaggle competition closes at **4:59pm on Friday February 23rd**

> Why does it say that the competition ends at 11:59pm when the assignment says 4:59pm? 

The end time/date are in UTC.  11:59pm UTC is equivalent to 4:59pm MST.  Kaggle In-Class does not allow us to change this. 